In [152]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import functions


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#We use eukaryota only

#Pick non sp from signalp5 training 

signalP = pd.read_csv('data/signalp5/train_set.fasta', \
                       sep='>', lineterminator='>', header=None)
signalP['Entry'] = signalP[0].apply(lambda x: x.split('|')[0])
signalP['Kingdom'] = signalP[0].apply(lambda x: x.split('|')[1])
signalP['Label'] = signalP[0].apply(lambda x: x.split('|')[2])
euk_signalp = signalP[signalP['Kingdom'].apply(lambda x: 'EUKARYA' in x)]
signalP['Accession'] = signalP[0].apply(lambda x: x.split('\n')[0])


#Select Eukarya
# signalP['Protein'] = signalP[0].apply(lambda x: x.split('\n')[1])
euk_signalp = signalP[signalP['Kingdom'] == 'EUKARYA'].copy()
euk_signalp.shape

(17270, 5)

In [3]:
euk_signalp.Label.value_counts()

NO_SP    14656
SP        2614
Name: Label, dtype: int64

In [4]:
export = euk_signalp[['Entry']]
export.to_csv('results/export_to_uniprot.fa', sep='\n', index=None, header=None)
##Uniprot id mapping to get sequences

In [5]:
#sequences
seqs = functions.read_seqs('results/signalp5/uniprot-yourlist M202011108471C63D39733769F8E060B506551E12040258H.fasta.gz')
#annotations
annots = pd.read_csv('results/signalp5/uniprot-yourlist M202011108471C63D39733769F8E060B506551E12040258H.tab.gz', sep='\t')

/Volumes/userdata/student_users/bikashkumarbhandari/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
all_seqs = pd.merge(seqs, annots, on='Entry')
all_seqs.shape

(17267, 39)

In [7]:
def cleavage(x):
    try:
        return int(x.split('..')[1].split(';')[0])
    except Exception:
        if str(x) != 'nan':
            print(x)
        return np.nan
    
all_seqs['Cleavage'] = all_seqs['Signal peptide'].apply(cleavage)
all_seqs = all_seqs[np.concatenate([all_seqs.columns[:3], all_seqs.columns[4:]])]

all_seqs = all_seqs.merge(euk_signalp[['Entry', 'Accession']], on='Entry')
all_seqs.head()

,Entry,All,Protein,Entry name,Status,Protein names,Gene names,Organism,Length,Signal peptide,...,Taxonomic lineage (SUPERFAMILY),Taxonomic lineage (SUPERKINGDOM),Cross-reference (Pfam),Biotechnological use,Motif,Domain [FT],Cross-reference (InterPro),Cross-reference (PROSITE),Cleavage,Accession
0,Q8TF40,sp|Q8TF40|FNIP1_HUMAN Folliculin-interacting p...,MAPTLFQKLFSKRTGLGAPGRDARDPDCGFSWPLPEFDPSQIRLIV...,FNIP1_HUMAN,reviewed,Folliculin-interacting protein 1,FNIP1 KIAA1961,Homo sapiens (Human),1166,NaN,...,Hominoidea (apes),Eukaryota,PF14638;PF14637;PF14636;,NaN,NaN,"DOMAIN 37..478; /note=""uDENN FNIP1/2-type""; ...",IPR037545;IPR028086;IPR026156;IPR028085;IPR028...,PS51836;,NaN,Q8TF40|EUKARYA|NO_SP|0
1,Q1ENB6,sp|Q1ENB6|FOLC_ARATH Folate synthesis bifuncti...,MDFTSLETTTFEEVVIALGSNVGNRMNNFKEALRLMKDYGISVTRH...,FOLC_ARATH,reviewed,Folate synthesis bifunctional protein [Include...,CytHPPK/DHPS At1g69190 F23O10.22 F4N2.15,Arabidopsis thaliana (Mouse-ear cress),484,NaN,...,NaN,Eukaryota,PF01288;PF00809;,NaN,NaN,"DOMAIN 202..470; /note=""Pterin-binding""; /ev...",IPR006390;IPR011005;IPR000550;IPR035907;IPR000...,PS00792;PS00793;PS00794;PS50972;,NaN,Q1ENB6|EUKARYA|NO_SP|1
2,P36001,sp|P36001|FOLC_YEAST Probable folylpolyglutama...,MDDISGRQTLPRINRLLEHVGNPQDSLSILHIAGTNGKETVSKFLT...,FOLC_YEAST,reviewed,Probable folylpolyglutamate synthase (EC 6.3.2...,RMA1 YKL132C,Saccharomyces cerevisiae (strain ATCC 204508 /...,430,NaN,...,NaN,Eukaryota,NaN,NaN,NaN,NaN,IPR001645;IPR018109;IPR036565;IPR036615;,PS01011;PS01012;,NaN,P36001|EUKARYA|NO_SP|1
3,P55317,sp|P55317|FOXA1_HUMAN Hepatocyte nuclear facto...,MLGTVKMEGHETSDWNSYYADTQEAYSSVPVSNMNSGLGSMNSMNT...,FOXA1_HUMAN,reviewed,Hepatocyte nuclear factor 3-alpha (HNF-3-alpha...,FOXA1 HNF3A TCF3A,Homo sapiens (Human),472,NaN,...,Hominoidea (apes),Eukaryota,PF00250;PF08430;PF09354;,NaN,NaN,NaN,IPR013638;IPR001766;IPR018533;IPR018122;IPR030...,PS00657;PS00658;PS50039;,NaN,P55317|EUKARYA|NO_SP|4
4,P35583,sp|P35583|FOXA2_MOUSE Hepatocyte nuclear facto...,MLGAVKMEGHEPSDWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSM...,FOXA2_MOUSE,reviewed,Hepatocyte nuclear factor 3-beta (HNF-3-beta) ...,Foxa2 Hnf3b Tcf-3b Tcf3b,Mus musculus (Mouse),459,NaN,...,NaN,Eukaryota,PF00250;PF08430;PF09354;,NaN,"MOTIF 106..113; /note=""Nuclear localization s...",NaN,IPR013638;IPR001766;IPR018533;IPR018122;IPR030...,PS00657;PS00658;PS50039;,NaN,P35583|EUKARYA|NO_SP|4


In [8]:
def chopping(a, p):
    '''
    Chops sequences at p position left and right of cleavage site a
    '''
    seq, cleavage = a.split('#')
    try:
        cleavage = int(float(cleavage))
    except Exception:
        print(cleavage)
    try:
        start = cleavage-p
    except Exception:
        print(cleavage, p)
    spaces = 0
    if start < 0:
        spaces = -1 * start
        start = 0
    try:
        s = seq[start:cleavage+p+1]
    except Exception:
        print(a, cleavage)
    return '_'* spaces + s

In [9]:
sp_ = all_seqs[~all_seqs['Signal peptide'].isna()]
# s = sp_[sp_['Signal peptide'].apply(lambda x: '0000269' in str(x))].copy()
s = sp_.copy()
s['Label'] = 1
set(sp_.Entry) - set(s.Entry)

set()

In [10]:
#Align signal peptides

s['Temp'] = s['Protein'] + '#' + s['Cleavage'].astype(str)


s['Aligned'] = s['Temp'].apply(lambda x: chopping(x, 20))
s['Aligned'].apply(len).value_counts()

41    2607
39       1
35       1
Name: Aligned, dtype: int64

In [11]:
all_seqs.columns

Index(['Entry', 'All', 'Protein', 'Entry name', 'Status', 'Protein names',
       'Gene names', 'Organism', 'Length', 'Signal peptide',
       'Taxonomic lineage IDs', 'Date of creation', 'Keywords',
       'Taxonomic lineage (all)', 'Taxonomic lineage (FAMILY)',
       'Gene ontology (biological process)', 'Transmembrane',
       'Allergenic properties', 'Toxic dose', 'Pharmaceutical use',
       'Involvement in disease', 'Domain [CC]', 'Gene ontology (GO)',
       'Gene ontology (molecular function)', 'Protein families',
       'Taxonomic lineage (CLASS)', 'Taxonomic lineage (COHORT)',
       'Taxonomic lineage (KINGDOM)', 'Taxonomic lineage (ORDER)',
       'Taxonomic lineage (SUPERCLASS)', 'Taxonomic lineage (SUPERFAMILY)',
       'Taxonomic lineage (SUPERKINGDOM)', 'Cross-reference (Pfam)',
       'Biotechnological use', 'Motif', 'Domain [FT]',
       'Cross-reference (InterPro)', 'Cross-reference (PROSITE)', 'Cleavage',
       'Accession'],
      dtype='object')

In [12]:
# P10184 is not SP

ns = all_seqs[all_seqs.Entry.apply(lambda x: x not in sp_.Entry.values)].copy()
ns['Label'] = 0
ns['Aligned'] = ns['Protein'].str[1:42]
ns.shape[0]

14655

In [13]:
columns = ['Entry', 'All', 'Protein', 'Entry name', 'Status', 'Protein names',
       'Gene names', 'Organism', 'Length', 'Signal peptide',
       'Taxonomic lineage IDs', 'Date of creation', 'Keywords',
       'Taxonomic lineage (all)', 'Taxonomic lineage (FAMILY)',
       'Gene ontology (biological process)', 'Transmembrane',
       'Allergenic properties', 'Toxic dose', 'Pharmaceutical use',
       'Involvement in disease', 'Domain [CC]', 'Gene ontology (GO)',
       'Gene ontology (molecular function)', 'Protein families',
       'Taxonomic lineage (CLASS)', 'Taxonomic lineage (COHORT)',
       'Taxonomic lineage (KINGDOM)', 'Taxonomic lineage (ORDER)',
       'Taxonomic lineage (SUPERCLASS)', 'Taxonomic lineage (SUPERFAMILY)',
       'Taxonomic lineage (SUPERKINGDOM)', 'Cross-reference (Pfam)',
       'Biotechnological use', 'Motif', 'Domain [FT]',
       'Cross-reference (InterPro)', 'Cross-reference (PROSITE)', 'Cleavage',
       'Accession', 'Label', 'Aligned']

final_df = pd.concat([s[columns], ns[columns]]).reset_index(drop=True)

In [14]:
final_df.Label.value_counts()

0    14655
1     2609
Name: Label, dtype: int64

In [15]:
final_df.to_pickle('results/signalp5_train_all.pkl.gz')

In [160]:

for_website = pd.read_pickle('results/signalp5_train_all.pkl.gz')
for_website.to_csv('results/SignalPeptide_Data.csv.gz', columns=['Accession', 'Protein', 'Label'],  index=None)

In [158]:

for_website.Protein

0        MARVAWGLLWLLLGSAGAQYEKYSFRGFPPEDLMPLAAAYGHALEQ...
1        MKAVITLLFLACILVVTYGDLICGTNYCKDHPCTSPIARASCRSPA...
2        MKIILTLSIFLICFLQLGQSVIDPSQNEVMSDLLFNLYGYDKSLDP...
3        MKHLLTLALCFSSINAVAVTVPHKAVGTGIPEGSLQFLSLRASAPI...
4        MLFKSLSKLATAAAFFAGVATADDVPAIEVVGNKFFYSNNGSQFYI...
                               ...                        
17259    MDTLRSISSELVRSSQLRWTLFSVIANMSLTLSIWIGVYNMEVNSE...
17260    MQGPAGNASRGLPGGPPSTVASGAGRCESGALMHSFGIFLQGLLGV...
17261    MDSPEKLEKNNLVGTNKSRLGVCGWILFFLSFLLMLVTFPISVWMC...
17262    MEELVGLREGSSGKPVTLQELWGPCPRIRRGIRGGLEWLKERLFRA...
17263    MEPHVLGAGLYWLLLPCTLLAASLLRFNALSLVYLLFLLLLPWLPG...
Name: Protein, Length: 17264, dtype: object

In [11]:
# chopping(all_seqs[all_seqs.Label == 1].loc[13612]['Protein'] + '#' + str(all_seqs[all_seqs.Label == 1].loc[13612]['Cleavage']), 10)

In [1]:
14655 + 2609

17264

### Benchmark set

In [16]:
benchmark = pd.read_csv('data/signalp5/benchmark_set.fasta', \
                       sep='>', lineterminator='>', header=None)
benchmark['Entry'] = benchmark[0].apply(lambda x: x.split('|')[0])
bm = final_df[final_df.Entry.apply(lambda x: x in benchmark.Entry.values)]

In [17]:
bm = bm.reset_index(drop=True)
bm.to_pickle('results/signalp5_benchmark.pkl.gz')

In [18]:
export = bm[['Entry', 'Protein']].copy()
export['Entry'] = '>' + export['Entry']
export.to_csv('results/benchmark.fa', index=False, header=None, sep='\n')

In [19]:
bm.Label.value_counts()

0    7246
1     211
Name: Label, dtype: int64

In [161]:

for_website = pd.read_pickle('results/signalp5_benchmark.pkl.gz')
for_website.to_csv('results/SignalPeptide_Benchmark.csv.gz', columns=['Accession', 'Protein', 'Label'],  index=None)